In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model


In [2]:
from federatedscope.contrib.data.cikm_cup import call_cikm_cup_data

register_data("cikm_cup", call_cikm_cup_data)

### Register model

In [3]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [5]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("federatedscope/gfl/baseline/laplacian_gine_on_cikmcup.yaml")
#cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
#cfg.trainer.type = 'laplacian_trainer'
#cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']

cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance=1e4
cfg.params.eps=1e-15
cfg.params.p=0.

#### configure other options

In [6]:
#cfg.use_gpu = True
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'
cfg.federate.method = \
    'Laplacian_batch_class_clients_GINE_global_jk_linear_24_10_22_csd_1e4_correct_csd'
#cfg.federate.local_update_steps = 20000000
#cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 20000

cfg.federate.client_num = 8
cfg.early_stop.patience = 20000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 5.0
#cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [7]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(False)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [8]:
from yacs.config import CfgNode
client_cfg_file = "federatedscope/gfl/baseline/laplacian_gine_on_cikmcup_per_client.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [9]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2022-10-25 11:24:03,650 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [10]:
from federatedscope.contrib.workers.laplacian_client import LaplacianClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer


setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-10-25 11:24:03,662 (utils:129)INFO: the current machine is at 127.0.1.1
2022-10-25 11:24:03,663 (utils:131)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-10-25 11:24:03,663 (utils:132)INFO: the output dir is exp/Laplacian_batch_class_clients_GINE_global_jk_linear_24_10_22_csd_1e4_correct_csd_gin_on_cikmcup_lr0.1_lstep10_/sub_exp_20221025112403
2022-10-25 11:24:03,725 (cikm_cup:57)INFO: Loading CIKMCUP data from /home/michael/Projects/CKIM_Competition/data/CIKM22Competition.
2022-10-25 11:24:03,726 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #1.
2022-10-25 11:24:04,204 (cikm_cup:67)INFO: Loading CIKMCUP data for Client #2.


In [11]:

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

server params: 
encoder_atom.atom_embedding_list.0.weight


2022-10-25 11:24:06,164 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-10-25 11:24:06,192 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 2
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: 

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
gnn.convs.0.nn.linears.0.weight
gnn.convs.0.nn.linears.0.bias
gnn.convs.0.nn.l

2022-10-25 11:24:06,376 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer.LaplacianTrainer'>
2022-10-25 11:24:06,377 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-10-25 11:24:06,378 (trainer:324)INFO: Model meta-info: <class 'federatedscope.gfl.model.graph_level.GNN_Net_Graph'>.
2022-10-25 11:24:06,380 (trainer:332)INFO: Num of original para names: 66.
2022-10-25 11:24:06,381 (trainer:333)INFO: Num of original trainable para names: 49.
2022-10-25 11:24:06,382 (trainer:335)INFO: Num of preserved para names in local update: 33. 
Preserved para names in local update: {'gnn.convs.0.nn.linears.0.bias', 'gnn.convs.1.nn.norms.1.bias', 'gnn.jk_linear.weight', 'gnn.convs.1.nn.norms.1.running_var', 'gnn.convs.1.nn.linears.1.bias', 'gnn.convs.1.nn.norms.1.weight', 'gnn.convs.0.nn.norms.0.bias', 'gnn.convs.0.nn.norms.1.weight', 'gnn.convs.0.nn.norms.1.bias', 'gnn.convs.0.nn.norms.1.num_batches_tracked', 'gnn.convs.1.

round num: 0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0


...

2022-10-25 11:25:40,374 (laplacian_client:118)INFO: {'Role': 'Client #1', 'Round': 4, 'Results_raw': {'train_loss': 2094.269243, 'train_avg_loss': 0.167676, 'train_imp_ratio': 79.846571, 'train_total': 12490}}


csd loss: 2.0448163922992535e-05
round num: 4
csd loss: 0.0
csd loss: 1.24722315808512e-10
csd loss: 2.2464413662603988e-10
csd loss: 3.128851622236084e-10
csd loss: 5.797046176425624e-10
csd loss: 7.572991678195251e-10
csd loss: 7.970752946562243e-10
csd loss: 1.1375285158266024e-09
csd loss: 1.49917522840326e-09
csd loss: 1.8279077140803679e-09
csd loss: 2.2256247955709796e-09
csd loss: 2.7268765023791275e-09
csd loss: 3.0396429817614035e-09
csd loss: 3.6091383215364203e-09
csd loss: 3.8768517285348025e-09
csd loss: 4.423034827283345e-09
csd loss: 4.878043746714411e-09
csd loss: 5.630525823363541e-09
csd loss: 6.131985585966504e-09
csd loss: 7.1333619011682e-09
csd loss: 7.854606742796477e-09
csd loss: 8.277038610060572e-09
csd loss: 9.45991285306036e-09
csd loss: 1.0085877022447676e-08
csd loss: 1.0541223893767437e-08
csd loss: 1.1396819488140864e-08
csd loss: 1.2706681928875696e-08
csd loss: 1.338338773138048e-08


2022-10-25 11:25:41,784 (laplacian_client:118)INFO: {'Role': 'Client #2', 'Round': 4, 'Results_raw': {'train_loss': 1118.456129, 'train_avg_loss': 0.617932, 'train_imp_ratio': -12.932536, 'train_total': 1810}}
2022-10-25 11:25:41,787 (server:480)INFO: {'Role': 'Server #', 'Round': 4, 'Results_avg': {'test_loss': 231.10054, 'test_avg_loss': 0.804262, 'test_imp_ratio': 6.595152, 'test_total': 239.0, 'val_loss': 73.668913, 'val_avg_loss': 0.448112, 'val_imp_ratio': 16.506397, 'val_total': 238.0}}
2022-10-25 11:25:41,788 (monitor:512)INFO: current_best=26.361867, should_save=False
2022-10-25 11:25:41,791 (monitor:512)INFO: current_best=26.361867, should_save=False
2022-10-25 11:25:41,843 (laplacian_server:160)INFO: Server #0: Starting evaluation at the end of round 4.
2022-10-25 11:25:41,846 (laplacian_server:167)INFO: ----------- Starting a new training round (Round #5) -------------


csd loss: 1.4484900390243638e-08
csd loss: 1.568413843244798e-08
round num: 5
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0
csd loss: 0.0


KeyboardInterrupt: 

In [ ]:
data

In [ ]:
import os

In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']